<a href="https://colab.research.google.com/github/yeon42/ml_ex/blob/main/garbage/garbe_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# API Token 다운받기
# 다운받은 API Token 업로드 하기
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
# json 파일 옮겨주기
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
ls -1ha kaggle.json

In [ ]:
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

In [ ]:
# 데이터셋 다운로드 받기 - 링크는 그 대회 'Data'에 있음
! kaggle datasets download -d asdasdasasdas/garbage-classification

In [ ]:
# 다운로드 된 것들 다 보기
!ls

In [ ]:
# 필요하면 압축 풀기 
!unzip garbage-classification.zip

In [ ]:
# 필요한 모듈 불러오기
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
# 데이터가 저장된 디렉토리
data_dir = './Garbage classification/Garbage classification/'

In [ ]:
# review number of files and directories in the dataset

total_dir = len(os.listdir(data_dir)) # 각 디렉토리의 길이 저장
total_files = 0

# os.walk: 시작 디렉터리에서 시작해 하위 모든 디렉터리를 차례대로 방문하는 함수
# for (path, dir, files) in os.walk("~")
for dirname, _, filenames in os.walk(data_dir):
    print('counting:', dirname) # 먼저 디렉토리 이름 출력
    files_counter = 0
    for file in filenames: # 각 디렉토리의 파일 갯수
        files_counter += 1
    total_files += files_counter # 전체 디렉토리의 총 파일 개수
    print('total files in dir:', files_counter) # 각 디렉토리의 파일 갯수

print('--------')
print('total number of files', total_files) # 전체 파일 갯수
print('total number of directories', total_dir) # 전체 디렉토리 갯수

In [ ]:
# 데이터셋 생성

# train 데이터셋 생성
# 클래스별로 폴더에 나뉜 이미지 파일을 -> 레이블된 이미지 텐서 데이터셋으로 변환
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, # 부동소수점(0-1): 검증의 용도로 남겨둘 이미지의 비율
    subset='training', # ImageDataGenerator에 validation_split이 설정된 경우 데이터의 부분세트
    seed=100 # 난수 시드
)

# validation 데이터셋 생성
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=100
)

"""
validation_split이 0.2이므로
train 세트는 0.8 / validation 세트는 0.2이다.
결과를 보면 비율도 4:1
"""

In [ ]:
# train셋의 클래스 이름
class_names = train_ds.class_names
print(class_names)

In [ ]:
# train셋의 일부 샘플 이미지 출력해보기 (랜덤)

plt.figure(figsize=(16, 16))

for images, labels in train_ds.take(1): # 배치 이미지 하나 가져오기
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) # 0-9의 정수
        plt.title(class_names[labels[i]])
        plt.axis("off") # 축 생략


# uint8 : 부호가 없는 8비트 (unsigned int 8bit, 0-255) 형식의 자료형

In [ ]:
# validation 셋의 일부 샘플 이미지 출력해보기 (랜덤)

plt.figure(figsize=(16, 16))

for images, labels in validation_ds.take(1):
    for i in range(12):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# next 메서드를 통해 train 데이터셋의 모든 요소를 읽어들임
train_batch = train_ds.as_numpy_iterator().next()

print('total of batches:', len(train_ds))

# 각 배치의 차원 출력(.shape)
print('images batch shape:', train_batch[0].shape) # 이미지 배치 차원
print('labels batch shape:', train_batch[1].shape) # 라벨 배치 차원

"""
딥러닝은 전체 데이터셋을 처리하지 않고 작은 배치(batch)로 나눈다.
"""

In [ ]:
# next 메서드를 통해 validation 데이터셋의 모든 요소를 읽어들임
validation_batch = validation_ds.as_numpy_iterator().next()

print('total of batches:',len(validation_ds))

# 각 배치의 차원 출력(.shape)
print('images batch shape:', validation_batch[0].shape)  # 이미지 배치 차원
print('labels batch shape:', validation_batch[1].shape)  # 라벨 배치 차원

"""
shape : (32,) => 32차원 벡터의 배치라는 것을 의미
      : (32, 256, 256, 3) => 256 x 256 픽셀의 3채널(컬러 이미지, 흑백은 1채널) 32차원 벡터의 배치
"""

In [ ]:
# 베이스 모델 
# 256 x 256 픽셀의 3채널 인풋 설정
input_shape = (256, 256, 3)

# RestNet50을 사용한 ImageNet 클래스 분류
base_model = tf.keras.applications.ResNet50V2(include_top=False, input_shape=input_shape)

"""
include_top : 네트워크 최상단에 완전 연결 레이어를 포함시킬지 여부
input_shape : 크기를 (256, 256, 3)으로 제한함
"""

# 베이스 모델 레이어가 훈련으로 업데이트되도록 설정
base_model.trainable = True # 기존 레이어의 가중치의 학습을 하겠다.

In [ ]:
# 베이스 모델 구조 요약해 출력
base_model.summary()

In [ ]:
tuning_layer_name = 'conv5_block1_preact_bn'  # 데이터셋 크기 조정할 레이어 이름 설정
tuning_layer = base_model.get_layer(tuning_layer_name)  # 베이스 모델의 레이어를 불러와 tuning 레이어로 설정
tuning_index = base_model.layers.index(tuning_layer)

# Tuning 과정에서 베이스 모델의 레이어에 학습이 반영되지 않도록 설정
for layer in base_model.layers[:tuning_index]:
    layer.trainable =  False

In [ ]:
# 이미지 분류 고도화 작업(데이터 전처리) 및 데이터 증가
# 데이터가 다양하지 않거나, 데이터 개수가 충분하지 않을 때 랜덤으로 회전시키거나 flip한느 등의 augmentation

data_augmentation = tf.keras.Sequential([ # data_augmentation: 캐라스에서 순차적 함수 만들고 이렇게 만들어진 합성 데이터를 사용해 데이터 양 늘리는 작업
    tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1),  # 이미지 데이터 값 스케일(범위) 재설정하기 # ex) (0,255) 범위를 (-1,1) 범위로 설정하기 위해서는 (1./127.5, offset=-1) 설정해야 함
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2), #( 0,2) 스케일 내에서 이미지 무작위 회전
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)  # 마찬가지 범위 내에서 무작위 확대
], name='data_augmentation')

In [ ]:
# 위에서 전처리 및 증량 작업 한 data_augmentation 가지고 실제 활용할 모델 생성
# Sequential: 레이어를 선형으로 연결하여 구성 (레이어 인스턴스를 생성자에게 넘겨줌으로써 Sequential 모델 구성 가능)

model = tf.keras.Sequential([       
    data_augmentation,
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),  # 레이어들의 차원을 줄이고 각 노드별 평균값을 아웃풋으로 제시
    tf.keras.layers.Dense(6, activation='softmax')  # dense = 연결강도 # softmax = 입력받은 값을 출력으로 0~1사이의 값으로 모두 정규화/출력 값들의 총합은 항상 1/분류하고 싶은 클래스 수만큼의 출력으로 구성
]) 

# 학습률 설정(작은 값부터 시작)
learning_rate = 0.00001

model.compile(
    loss='sparse_categorical_crossentropy',      # compile을 위한 손실함수 설정
    optimizer=tf.keras.optimizers.Adam(lr=learning_rate),  # 함수 최적화하기 (학습률 유지하면서)
    metrics=['accuracy']                              # 정확도 판정
)

In [ ]:
# 모델 훈련시키기
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=15
)

In [ ]:
# 시각화 작업
# 지금까지 한 것들(history) 전부 시각화
acc = history.history['accuracy']  
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

# plot accuracy(훈련 정확도) 시각화하기
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')  # 하나의 그래프 안에 데이터가 모두 위치하도록 설정(legend함수)
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

# plot loss
plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# 모델 성능을 검증해 본다.
loss, accuracy = model.evaluate(validation_ds)
print('Test accuracy :', accuracy)
print('Test loss:', loss)

In [ ]:
# 이전에 학습했던 데이터로 추론해 보자.
# 이전에 학습했던 validation 데이터셋에서 batch를 가져온다.
image_batch, label_batch = validation_ds.as_numpy_iterator().next()

# 추론결과 저장
inference = model.predict_on_batch(image_batch)

# 이미지들과 라벨을 보여준다.
plt.figure(figsize=(18, 18))
for i in range(12):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title('Inference:{}, {:.2f}% Confidence\nReal Label:{}'
              .format(class_names[np.argmax(inference[i])], 100 * np.max(inference[i]), class_names[label_batch[i]]))
    plt.axis("off")

In [ ]:
# 이전에 학습해 보지 않은 새로운 데이터로 추론해 보자.

# 각 레이블을 사용해 테스트할 이미지 url들을 정의해 준다.
test_urls = [
    'https://uploads.ifdesign.de/award_img_121/oex_large/31983_01_4078_basket.jpg',
    'https://www.antique-bottles.net/attachments/image-jpg.201847/',
    'https://inzanetimes.files.wordpress.com/2013/04/canterbury41313016.jpg',
    'https://live.staticflickr.com/66/167934943_f61a850d96_b.jpg',
    'https://cdnimg.webstaurantstore.com/images/products/large/407128/1501927.jpg',
    'https://dy6g3i6a1660s.cloudfront.net/6n3yAQDV3zPhUIdvbTC-uwPUAoo/orig.jpg'
]
test_labels = [3, 1, 2, 4, 0, 5]

# test 데이터셋을 만들어서 확인
test_ds = []
image_size = (256, 256)
for i in range(len(test_urls)):
    path = tf.keras.utils.get_file(str(i), origin=test_urls[i])
    img = tf.keras.preprocessing.image.load_img(path, target_size=image_size)
    test_ds.append(tf.keras.preprocessing.image.img_to_array(img))
test_ds = np.array(test_ds)

# test 추론 결과 저장
test_inference = model.predict_on_batch(test_ds)

# 이미지들과 라벨을 보여준다.
plt.figure(figsize=(16, 16))
for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(test_ds[i].astype("uint8"))
    plt.title('Inference:{}, {:.2f}% Confidence\nReal Label:{}'
              .format(class_names[np.argmax(test_inference[i])], 100 * np.max(test_inference[i]), class_names[test_labels[i]]))
    plt.axis("off")